<a href="https://colab.research.google.com/github/Praise-Atadja/EIICD_chatbox/blob/main/CognitiveQuest_chatbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **PROJECT NAME:**

##EARLY INTERVENTION INTELLIGENCE FOR COGNITIVE DEVELOPMENT (EIICD) CHATBOX


---

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install transformers

In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
import nltk
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer,  DataCollatorWithPadding, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
import tensorflow_hub as hub
import pandas as pd
import re
from sklearn.metrics import classification_report
import joblib
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

## THE DATASET

In [ ]:
from datasets import load_dataset

# Load the dataset from the CSV file
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Autism Articles/Autsim_Q&A_datatset.csv')

dataset

In [ ]:
# Split the dataset into training and validation sets
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
val_dataset = dataset['test']

In [ ]:
# Display the number of examples in the training and validation sets
print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of validation examples: {len(val_dataset)}")

In [ ]:
print(train_dataset[:5])
print(val_dataset[:5])

In [ ]:
dataset["train"][0]

In [ ]:
import json

# Convert Dataset objects to lists of dictionaries
dataset_dict = [example for example in train_dataset]
train_data_dict = [example for example in train_dataset]
val_data_dict = [example for example in val_dataset]

# Define file paths
train_file = 'train_dataset.json'
val_file = 'val_dataset.json'
dataset_file = 'dataset.json'

# Save training dataset to JSON
with open(train_file, 'w') as f:
    json.dump(train_data_dict, f, indent=4)

# Save validation dataset to JSON
with open(val_file, 'w') as f:
    json.dump(val_data_dict, f, indent=4)

# Save dataset to JSON
with open(dataset_file, 'w') as f:
    json.dump(dataset_dict, f, indent=4)

print(f"Training dataset saved to {train_file}")
print(f"Validation dataset saved to {val_file}")

In [ ]:
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and digits
    text = ' '.join(text.split())  # Remove extra whitespaces
    return text

In [ ]:
from transformers import AutoTokenizer

# Define your tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_dataset(dataset):
    tokenized_dataset = []

    # Iterate through each sample in the dataset
    for sample in dataset:
        tokenized_example = tokenizer(
            sample["Questions"],
            sample["Patterns"],
            padding=True,
            truncation=True,
            max_length=512,  # Adjust this based on your requirements
            return_tensors="pt"  # Return PyTorch tensors
        )
        tokenized_dataset.append(tokenized_example)

    return tokenized_dataset

# Assuming train_dataset is a list of dictionaries where each dictionary represents a sample
tokenized_train_dataset = tokenize_dataset(train_dataset)
tokenized_val_dataset = tokenize_dataset(val_dataset)

In [ ]:
# Save tokenized train dataset to a JSON file
train_tokenized_file = 'train_tokenized_dataset.json'

# Convert tokenized dataset to JSON serializable format
tokenized_train_serializable = []

for tokenized_example in tokenized_train_dataset:
    # Convert BatchEncoding object to dictionary
    tokenized_example_dict = {
        "input_ids": tokenized_example["input_ids"].tolist(),
        "attention_mask": tokenized_example["attention_mask"].tolist(),
        # Add other relevant keys if necessary
    }
    tokenized_train_serializable.append(tokenized_example_dict)

# Save tokenized dataset to JSON
with open(train_tokenized_file, 'w') as f:
    json.dump(tokenized_train_serializable, f, indent=2)

print(f"Tokenized training dataset saved to {train_tokenized_file}")


In [ ]:
# Define your tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_dataset(dataset):
    tokenized_dataset = []

    for sample in dataset:
        # Assuming 'label' key exists in your original dataset
        label = sample.get('label')
        if label is None:
            label = -1  # Or any other default value that makes sense in your context
        tokenized_example = tokenizer(
            sample["Questions"],
            sample["Patterns"],
            padding='max_length',  # Ensure consistent sequence lengths
            truncation=True,  # Handle long sequences
            max_length=512,  # Adjust this based on your requirements
        )
        tokenized_example['label'] = label  # Add the label to the tokenized example
        tokenized_dataset.append(tokenized_example)

    return tokenized_dataset

# Assuming train_dataset and val_dataset are lists of dictionaries where each dictionary represents a sample
tokenized_train_dataset = tokenize_dataset(train_dataset)
tokenized_val_dataset = tokenize_dataset(val_dataset)

# Function to convert tokenized dataset to PyTorch tensors
def collate_fn(batch):
    input_ids = torch.tensor([example['input_ids'] for example in batch])
    attention_mask = torch.tensor([example['attention_mask'] for example in batch])
    labels = torch.tensor([example['label'] for example in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",  # Output directory
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    num_train_epochs=3,
)

# Use DataCollatorWithPadding to handle variable sequence lengths
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize the model
num_labels = 2  # Replace with the number of classes in your dataset
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# Convert tokenized datasets to PyTorch Datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.tokenized_data[idx].items()}

train_dataset = CustomDataset(tokenized_train_dataset)
val_dataset = CustomDataset(tokenized_val_dataset)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()


In [ ]:
# Function for semantic search using Universal Sentence Encoder (USE)
# Load the Universal Sentence Encoder model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def semantic_search(corpus, query):
    query_embedding = use_model([query])[0]
    corpus_embeddings = use_model(corpus)
    similarity = np.inner(query_embedding, corpus_embeddings)
    closest = np.argmax(similarity)
    return corpus[closest]

# Example usage of semantic search
df = pd.read_csv('/content/drive/MyDrive/AutsimChatbox_datatset.csv')
corpus = df['Patterns'].apply(preprocess_text).tolist()
query = "How does autism affect sleep patterns?"

most_similar_text = semantic_search(corpus, query)
print("Most similar text:", most_similar_text)

MODEL TRAINING

In [ ]:
class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# Convert datasets to the QADataset format
train_dataset = QADataset(train_dataset)
val_dataset = QADataset(val_dataset)

# Initialize the model
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()



EVALUATING THE MODEL

In [ ]:
# Save the model and tokenizer
model_save_path = './saved_model'
tokenizer_save_path = './saved_tokenizer'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to {model_save_path}")
print(f"Tokenizer saved to {tokenizer_save_path}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Function to get answer from model
def get_answer(question, context, model, tokenizer):
    # Tokenize inputs
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', truncation=True)

    # Get input IDs and attention mask
    input_ids = inputs['input_ids'].tolist()[0]
    attention_mask = inputs['attention_mask']

    # Get the model's outputs
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Get the most likely beginning and end of answer with the argmax of the score
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    # Convert tokens to answer text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer

# Load the model and tokenizer
model_save_path = './saved_model'
tokenizer_save_path = './saved_tokenizer'

model = AutoModelForQuestionAnswering.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)

# Example usage
question = "What are some common symptoms of autism?"
context = (" 'What are typical autism symptoms?.")

answer = get_answer(question, context, model, tokenizer)
print(f"Question: {question}")
print(f"Answer: {answer}")
